In [46]:
import json
import openai
import os
import dotenv
dotenv.load_dotenv()

True

In [47]:
client = openai.Client(api_key=os.environ["openaiKey"])

In [48]:
def saudacao_por_periodo(hora):
    if 5 <= 5 < 12:
        return json.dumps({"saudacao":"bom dia"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao":"boa tarde"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao":"boa noite"})
    else:
        return json.dumps({"saudacao":"boa madrugada"})

In [49]:
tools = [{"type":"function"
          ,"function":{"name":"saudacao_por_periodo",
                       "description":"retorna uma saudação baseada na hora do dia"}
          ,"parameters":{"type":"object",
                         "properties":{"hora":{"type":"integer",
                                               "description":"a hora do dia em formato de 24h"}},"required":["hora"]}}]

In [50]:
funcao_disponivel = {"saudacao_por_periodo":saudacao_por_periodo}

In [51]:
mensagens = [{"role":"user","content":"qual saudação o modelo me da se for 10h?"}]

In [52]:
resposta = client.chat.completions.create(model="gpt-3.5-turbo-0125",messages=mensagens,tools=tools,tool_choice="auto")

In [53]:
mensagem = resposta.choices[0].message
tool_calls = mensagem.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_EyCnpJyaPeyVZNDKHQPLi8ro', function=Function(arguments='{}', name='saudacao_por_periodo'), type='function')]

In [54]:
if tool_calls:
    mensagens.append(mensagem)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(hora=function_args.get("hora"))

        mensagens.append({"tool_call_id":tool_call.id,"role":"tool","name":function_name,"content":function_response})


    segunda_resposta = client.chat.completions.create(model="gpt-3.5-turbo-0125",messages=mensagens)    


In [55]:
mensagem = segunda_resposta.choices[0].message
mensagem

ChatCompletionMessage(content='Bom dia!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)